# Module 05 — Semiconductor Zoo
# Notebook 02: Sensors
## Hall Effect Sensors, Photoresistors, Varactors

---

Semiconductors can sense the physical world — magnetic fields, light, temperature,
and more. This notebook explores sensors that convert physical phenomena into
electrical signals, with a focus on the physics that makes each one work.

---
## Concept — The Hall Effect

When current flows through a conductor (or semiconductor) and a **magnetic field**
is applied perpendicular to the current, a voltage appears across the material
perpendicular to *both* the current and the field. This is the **Hall effect**,
discovered by Edwin Hall in 1879.

### Why it happens:

1. Charge carriers (electrons or holes) flow through the material
2. The magnetic field exerts a **Lorentz force** on them: $\vec{F} = q\vec{v} \times \vec{B}$
3. Carriers are deflected to one side of the material
4. Charge builds up on that side, creating a **Hall voltage**
5. Equilibrium: the electric field from charge buildup balances the magnetic force

$$V_H = \frac{I \cdot B}{n \cdot q \cdot t}$$

where:
- $V_H$ — Hall voltage
- $I$ — current through the sensor
- $B$ — magnetic field strength (Tesla)
- $n$ — carrier concentration
- $q$ — electron charge
- $t$ — thickness of the material

Semiconductors make better Hall sensors than metals because they have **lower carrier
concentration** ($n$), which gives a *larger* Hall voltage for the same field.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- Hall Voltage vs Magnetic Field Strength ---

# AH3503 linear Hall sensor typical parameters:
# Sensitivity: ~1.3 mV/Gauss at 5V supply
# Quiescent output: ~2.5V (at zero field)
# Output range: ~0.5V to 4.5V

sensitivity = 1.3e-3  # V per Gauss
V_q = 2.5             # quiescent output voltage (V)

# Magnetic field range: -1500 to +1500 Gauss (typical range)
B_gauss = np.linspace(-1500, 1500, 500)
V_out = V_q + sensitivity * B_gauss

# Clip to supply rails
V_out = np.clip(V_out, 0.2, 4.8)

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(B_gauss, V_out, 'b-', linewidth=2)
ax.axhline(y=V_q, color='gray', linestyle='--', alpha=0.5, label='Zero field (2.5V)')
ax.axvline(x=0, color='gray', linestyle=':', alpha=0.5)

# Mark some key points
for B_mark in [-500, 0, 500, 1000]:
    V_mark = V_q + sensitivity * B_mark
    V_mark = np.clip(V_mark, 0.2, 4.8)
    ax.plot(B_mark, V_mark, 'ro', markersize=8)
    ax.annotate(f'{V_mark:.2f}V', (B_mark, V_mark),
                textcoords='offset points', xytext=(10, 10), fontsize=9)

ax.set_xlabel('Magnetic Field (Gauss)', fontsize=12)
ax.set_ylabel('Output Voltage (V)', fontsize=12)
ax.set_title('AH3503 Hall Effect Sensor Output vs Magnetic Field', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_xlim(-1500, 1500)
ax.set_ylim(0, 5)

# Add N/S pole indicators
ax.text(-1400, 0.5, 'South Pole\n(toward face)', fontsize=9, color='blue',
        ha='left', style='italic')
ax.text(1400, 0.5, 'North Pole\n(toward face)', fontsize=9, color='red',
        ha='right', style='italic')

plt.tight_layout()
plt.show()

print('AH3503 Key Specs:')
print(f'  Supply: 5V')
print(f'  Sensitivity: {sensitivity*1000:.1f} mV/Gauss')
print(f'  Quiescent output: {V_q} V (no field)')
print(f'  South pole -> voltage decreases')
print(f'  North pole -> voltage increases')

### Hall Effect Applications

- **Current sensing**: Clamp a Hall sensor around a wire; the magnetic field
  is proportional to current ($B = \mu_0 I / 2\pi r$). No need to break the circuit.
- **Position sensing**: Detect magnet proximity (door sensors, lid switches)
- **Speed sensing**: Count magnets on a rotating shaft (RPM measurement)
- **Brushless DC motors**: Hall sensors detect rotor position for commutation
- **Joysticks and throttles**: Linear position measurement

---
## The Material Science Why — CdS Photoresistors (LDRs)

A **Cadmium Sulfide (CdS) photoresistor** (also called LDR — Light Dependent Resistor)
changes resistance based on light intensity.

### How it works:

CdS is a semiconductor with a bandgap of about 2.4 eV (corresponding to green
light at ~520 nm). When photons with enough energy hit the CdS:

1. Photons excite electrons from valence band to conduction band
2. More light = more free carriers = lower resistance
3. Dark: very few carriers, resistance is 200k–1M ohm
4. Bright: many carriers, resistance drops to 100–500 ohm

The relationship is approximately:

$$R = R_0 \cdot E^{-\gamma}$$

where $E$ is illuminance (lux) and $\gamma \approx 0.7$ to $0.9$ (the gamma coefficient).
This is why resistance vs light plots are best shown on **log-log** scales.

### Limitations:
- **Slow response**: Rise time ~20 ms, fall time ~30 ms (trapped carriers release slowly)
- **Spectral response**: Peaks in green (~550 nm), poor in red/IR
- **Temperature sensitive**: Resistance drifts with temperature
- **Contains cadmium**: Restricted by RoHS in some applications

In [ ]:
# --- Photoresistor Resistance vs Light Intensity (Log-Log) ---

# Typical CdS LDR characteristics
lux = np.logspace(-1, 4, 500)  # 0.1 to 10,000 lux

# R = R_10 * (lux/10)^(-gamma)
R_10 = 10_000   # resistance at 10 lux (kohm range varies by part)
gamma = 0.8      # gamma coefficient

R = R_10 * (lux / 10) ** (-gamma)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Log-log plot
ax1.loglog(lux, R, 'b-', linewidth=2)

# Mark typical lighting conditions
conditions = [
    (0.5, 'Moonlight'),
    (50, 'Dim room'),
    (300, 'Office'),
    (1000, 'Overcast'),
    (10000, 'Direct sun'),
]
for lx, label in conditions:
    r = R_10 * (lx / 10) ** (-gamma)
    ax1.plot(lx, r, 'ro', markersize=8)
    ax1.annotate(f'{label}\n{r:.0f} ohm' if r < 1000 else f'{label}\n{r/1000:.1f} kohm',
                 (lx, r), textcoords='offset points', xytext=(10, 10), fontsize=8)

ax1.set_xlabel('Illuminance (lux)', fontsize=12)
ax1.set_ylabel('Resistance (ohm)', fontsize=12)
ax1.set_title('CdS Photoresistor: Resistance vs Light', fontsize=13)
ax1.grid(True, which='both', alpha=0.3)

# Spectral response
wavelength = np.linspace(300, 800, 500)
# Approximate CdS spectral response (peaks ~550 nm)
response = np.exp(-0.5 * ((wavelength - 550) / 80) ** 2)
# Human eye response (photopic)
eye_response = np.exp(-0.5 * ((wavelength - 555) / 65) ** 2)

ax2.plot(wavelength, response, 'g-', linewidth=2, label='CdS photoresistor')
ax2.plot(wavelength, eye_response, 'k--', linewidth=1.5, alpha=0.5, label='Human eye (photopic)')
ax2.fill_between(wavelength, response, alpha=0.1, color='green')

# Color bands
color_bands = [
    (380, 450, '#8A2BE2', 'Violet'),
    (450, 495, '#0000FF', 'Blue'),
    (495, 570, '#00FF00', 'Green'),
    (570, 590, '#FFFF00', 'Yellow'),
    (590, 620, '#FFA500', 'Orange'),
    (620, 700, '#FF0000', 'Red'),
]
for wl_min, wl_max, color, name in color_bands:
    ax2.axvspan(wl_min, wl_max, alpha=0.05, color=color)

ax2.set_xlabel('Wavelength (nm)', fontsize=12)
ax2.set_ylabel('Relative Response', fontsize=12)
ax2.set_title('CdS Spectral Response vs Human Eye', fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# --- Interactive: LDR Voltage Divider ---

import ipywidgets as widgets
from IPython.display import display

def plot_ldr_divider(light_lux=300, R_fixed_kohm=10.0, Vcc=5.0):
    """
    Voltage divider with CdS LDR on top, fixed resistor on bottom.
    Shows output voltage for given light level.
    """
    R_10 = 10_000  # ohms at 10 lux
    gamma = 0.8

    # Current LDR resistance
    R_ldr = R_10 * (light_lux / 10) ** (-gamma)
    R_fixed = R_fixed_kohm * 1000  # convert to ohms

    V_out = Vcc * R_fixed / (R_ldr + R_fixed)

    # Sweep across light levels
    lux_sweep = np.logspace(-1, 4, 300)
    R_sweep = R_10 * (lux_sweep / 10) ** (-gamma)
    V_sweep = Vcc * R_fixed / (R_sweep + R_fixed)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Output voltage vs light
    ax1.semilogx(lux_sweep, V_sweep, 'b-', linewidth=2)
    ax1.plot(light_lux, V_out, 'ro', markersize=12,
             label=f'{light_lux} lux: Vout={V_out:.2f}V')
    ax1.axhline(y=Vcc/2, color='gray', linestyle='--', alpha=0.4, label=f'Vcc/2 = {Vcc/2:.1f}V')

    ax1.set_xlabel('Light Intensity (lux)', fontsize=12)
    ax1.set_ylabel('Output Voltage (V)', fontsize=12)
    ax1.set_title(f'LDR Voltage Divider (R_fixed={R_fixed_kohm:.1f}k)', fontsize=13)
    ax1.legend(fontsize=10)
    ax1.grid(True, which='both', alpha=0.3)
    ax1.set_ylim(0, Vcc + 0.2)

    # Status display
    ax2.axis('off')
    info = (f"LDR Voltage Divider\n"
            f"{'='*30}\n\n"
            f"Vcc:       {Vcc:.1f} V\n"
            f"R_fixed:   {R_fixed_kohm:.1f} kohm\n"
            f"Light:     {light_lux:.0f} lux\n"
            f"R_LDR:     {R_ldr:.0f} ohm ({R_ldr/1000:.2f} kohm)\n\n"
            f"V_out = {Vcc:.1f} x {R_fixed:.0f} / ({R_ldr:.0f} + {R_fixed:.0f})\n"
            f"V_out = {V_out:.3f} V\n\n"
            f"More light -> lower R_LDR -> higher V_out")

    ax2.text(0.1, 0.5, info, fontsize=12, fontfamily='monospace',
             verticalalignment='center', transform=ax2.transAxes,
             bbox=dict(boxstyle='round', facecolor='lightyellow', edgecolor='gray'))

    plt.tight_layout()
    plt.show()

widgets.interact(
    plot_ldr_divider,
    light_lux=widgets.FloatLogSlider(value=300, base=10, min=-1, max=4, step=0.1,
                                      description='Light (lux)'),
    R_fixed_kohm=widgets.FloatSlider(value=10.0, min=1.0, max=100.0, step=1.0,
                                      description='R_fixed (kohm)'),
    Vcc=widgets.FloatSlider(value=5.0, min=3.0, max=12.0, step=0.1,
                             description='Vcc (V)')
);

---
## Concept — Varactor Diodes

A **varactor** (varicap) diode exploits the fact that a reverse-biased PN junction
acts as a voltage-dependent capacitor.

### Why the capacitance changes:

Recall from Module 02: the **depletion region** in a PN junction contains no free
carriers — it acts like the dielectric of a capacitor, with the P and N regions
acting as the plates.

$$C_j = \frac{C_0}{\left(1 + \frac{V_R}{V_0}\right)^{m}}$$

where:
- $C_0$ — zero-bias capacitance
- $V_R$ — reverse bias voltage
- $V_0$ — built-in potential (~0.7V for silicon)
- $m$ — grading coefficient (0.5 for abrupt junction, ~0.3–0.5 typical)

**More reverse bias = wider depletion region = less capacitance**

### Applications:
- **Voltage-Controlled Oscillators (VCOs)**: Vary frequency by changing varactor bias
- **FM modulation**: Audio signal modulates varactor -> frequency deviation
- **Electronic tuning**: Replace mechanical tuning capacitors in radios
- **Phase-locked loops (PLLs)**: VCO is the heart of a PLL

In [ ]:
# --- Varactor Capacitance vs Reverse Voltage ---

V_reverse = np.linspace(0, 20, 500)

# Typical varactor parameters (e.g., BB910 or similar)
C_0 = 30e-12      # 30 pF at zero bias
V_0 = 0.7         # built-in potential
m_values = [0.3, 0.5, 0.7]  # different grading coefficients

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for m in m_values:
    C = C_0 / (1 + V_reverse / V_0) ** m
    ax1.plot(V_reverse, C * 1e12, linewidth=2, label=f'm = {m}')

ax1.set_xlabel('Reverse Voltage (V)', fontsize=12)
ax1.set_ylabel('Capacitance (pF)', fontsize=12)
ax1.set_title('Varactor Capacitance vs Reverse Bias', fontsize=13)
ax1.legend(fontsize=10, title='Grading coeff.')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 20)

# Tuning ratio
m = 0.5  # typical abrupt junction
C = C_0 / (1 + V_reverse / V_0) ** m

# If this varactor is in an LC oscillator: f = 1/(2*pi*sqrt(L*C))
L = 10e-6  # 10 uH inductor
f = 1 / (2 * np.pi * np.sqrt(L * C))

ax2.plot(V_reverse, f / 1e6, 'r-', linewidth=2)
ax2.set_xlabel('Varactor Reverse Voltage (V)', fontsize=12)
ax2.set_ylabel('Oscillator Frequency (MHz)', fontsize=12)
ax2.set_title(f'VCO Frequency (L = {L*1e6:.0f} uH)', fontsize=13)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 20)

# Mark tuning range
f_min = f[0] / 1e6
f_max = f[-1] / 1e6
ax2.axhspan(f_min, f_max, alpha=0.1, color='red')
ax2.annotate(f'Tuning range:\n{f_min:.1f} - {f_max:.1f} MHz\nRatio: {f_max/f_min:.2f}:1',
             (10, (f_min + f_max) / 2), fontsize=10,
             bbox=dict(boxstyle='round', facecolor='lightyellow'))

plt.tight_layout()
plt.show()

---
## Concept — Thermistors and Semiconductor Temperature Sensors

### NTC Thermistors (Negative Temperature Coefficient)

Made from semiconductor metal oxides. As temperature increases:
- More electrons get thermally excited across the bandgap
- More carriers = lower resistance
- Resistance decreases exponentially: $R = R_0 \cdot e^{B(1/T - 1/T_0)}$

### PTC Thermistors (Positive Temperature Coefficient)

Made from doped barium titanate ceramics. Above a critical temperature:
- Crystal structure changes, dramatically increasing resistance
- Used as resettable fuses (polyfuses)

### Semiconductor Temperature Sensors

Devices like the LM35 or TMP36 use the temperature dependence of
a PN junction's forward voltage:

$$V_f \approx V_0 - \alpha \cdot T$$

where $\alpha \approx -2$ mV/C. By processing this inside an IC, you get
a calibrated, linear output.

| Sensor Type | Output | Range | Accuracy | Cost |
|-------------|--------|-------|----------|------|
| NTC Thermistor | Resistance | -40 to 150C | +/-1C with cal. | Very low |
| PTC Thermistor | Resistance | Switch-like | N/A | Low |
| LM35/TMP36 | Voltage (10mV/C) | -40 to 125C | +/-1C | Low |
| Thermocouple | Voltage (uV/C) | -200 to 1800C | +/-1-2C | Medium |

In [ ]:
# --- NTC Thermistor Resistance vs Temperature ---

# Steinhart-Hart / B-parameter model
R_25 = 10_000   # 10k at 25C (common NTC value)
B = 3950         # B-parameter (typical for 10k NTC)
T_0 = 298.15     # 25C in Kelvin

T_C = np.linspace(-20, 120, 500)
T_K = T_C + 273.15

R_ntc = R_25 * np.exp(B * (1/T_K - 1/T_0))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.semilogy(T_C, R_ntc, 'b-', linewidth=2)
ax1.plot(25, R_25, 'ro', markersize=10, label=f'25C: {R_25/1000:.0f} kohm')

# Mark some key temps
for t in [0, 25, 50, 75, 100]:
    r = R_25 * np.exp(B * (1/(t+273.15) - 1/T_0))
    ax1.plot(t, r, 'ko', markersize=6)
    if r > 1000:
        ax1.annotate(f'{r/1000:.1f}k', (t, r), textcoords='offset points',
                     xytext=(10, 0), fontsize=9)
    else:
        ax1.annotate(f'{r:.0f}', (t, r), textcoords='offset points',
                     xytext=(10, 0), fontsize=9)

ax1.set_xlabel('Temperature (C)', fontsize=12)
ax1.set_ylabel('Resistance (ohm)', fontsize=12)
ax1.set_title('10k NTC Thermistor (B=3950)', fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, which='both', alpha=0.3)

# LM35 / TMP36 linear output
T_lm35 = np.linspace(-40, 125, 300)
V_lm35 = T_lm35 * 10e-3   # 10 mV/C, 0V at 0C
V_tmp36 = (T_lm35 + 50) * 10e-3  # 10 mV/C, 0.5V at 0C (offset for negative temps)

ax2.plot(T_lm35, V_lm35, 'b-', linewidth=2, label='LM35 (10 mV/C, 0V at 0C)')
ax2.plot(T_lm35, V_tmp36, 'r-', linewidth=2, label='TMP36 (10 mV/C, 0.75V at 25C)')
ax2.axhline(y=0, color='black', linewidth=0.5)
ax2.set_xlabel('Temperature (C)', fontsize=12)
ax2.set_ylabel('Output Voltage (V)', fontsize=12)
ax2.set_title('Semiconductor Temperature Sensors', fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## Experiment 1 — AH3503 Hall Effect Sensor

### Equipment
- Breadboard and jumper wires
- AH3503 linear Hall effect sensor (SIP-3 package)
- Bench power supply set to 5V
- Klein MM300 multimeter
- Small neodymium magnet

### Circuit

```
  +5V ────────────────── Pin 1 (Vcc)
                              
       ┌──────────┐          
       │  AH3503  │     (flat side facing you)
       │          │     Pin 1: Vcc (left)
       │ 1  2  3  │     Pin 2: GND (center)
       └──┬──┬──┬─┘     Pin 3: Output (right)
          │  │  │
          │  │  └──── V_out ──── Multimeter (+)
          │  │
          │  └──── GND
          │
          └──── +5V

  Multimeter: DC Voltage between Pin 3 and GND
```

**Procedure:**
1. Wire the AH3503 to 5V power
2. Set multimeter to DC Voltage
3. Measure output voltage with no magnet nearby — should read ~2.5V
4. Bring the North pole of a magnet toward the sensor face — voltage should increase
5. Flip the magnet (South pole) — voltage should decrease
6. Try different distances and note the voltage change

**Expected:** Output varies roughly linearly with field strength, centered at 2.5V.
A small neodymium magnet at 1 cm might produce 100–500 Gauss,
giving a few hundred mV of change.

---
## Experiment 2 — CdS Photoresistor Measurements

### Circuit: Direct Resistance Measurement

```
  ┌──────────────┐
  │  CdS LDR     │
  │  (no power)  │
  └──┬────────┬──┘
     │        │
    V+       COM    Klein MM300 set to OHMS
```

**Procedure:**
1. Set multimeter to resistance (ohms)
2. Connect leads to the photoresistor (polarity does not matter)
3. Measure resistance in several conditions:

| Condition | Expected Resistance |
|-----------|--------------------|
| Complete dark (cover with hand + cloth) | 200k – 1M ohm |
| Dim room | 20k – 50k ohm |
| Normal room lighting | 2k – 10k ohm |
| Phone flashlight directly on it | 100 – 500 ohm |

---
## Experiment 3 — Light Level Indicator

Build a circuit that lights an LED when it gets dark.

```
  +5V
   │
   ┌────────┐
   │ 10k    │  (fixed resistor)
   └───┬────┘
       │
       ├───────── V_mid (junction point)
       │
   ┌───┴────┐
   │  CdS   │  (photoresistor)
   │  LDR   │
   └───┬────┘
       │
      GND

  V_mid drives an LED through a 330 ohm resistor:

  V_mid ──┬──[330 ohm]──►│── GND
          │               LED
```

**How it works:**
- In bright light: LDR is low resistance, V_mid is LOW -> LED off
- In dark: LDR is high resistance, V_mid is HIGH -> LED on

Note: The fixed resistor is on top here (opposite from Experiment 2 in notebook 00)
so that voltage goes UP when it gets dark. This is a classic "nightlight" circuit.

**Challenge:** The LED turn-on is not sharp because it is driven by analog voltage.
In Module 06, we will use a comparator or Schmitt trigger for a clean on/off threshold.

---
## Simulation — Build It in Falstad

### LDR (Light-Dependent Resistor) Voltage Divider

**What to build:**
1. Open [Falstad (blank canvas)](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Build a voltage divider: 5V → fixed resistor (10 kΩ) → variable resistor → ground
3. The variable resistor simulates the LDR — use a potentiometer or simply change the bottom resistor's value manually

**What to observe:**
- In bright light, an LDR has low resistance (~1 kΩ) → output voltage is low
- In darkness, an LDR has high resistance (~1 MΩ) → output voltage is close to 5V
- Try resistance values of 1 kΩ, 10 kΩ, 100 kΩ, and 1 MΩ and note how V_out changes
- This is how light sensors work: light changes resistance, which changes voltage, which a microcontroller can read

### Varactor-Tuned Oscillator

**What to build:**
1. Open a [new blank canvas](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Build an LC oscillator: place an inductor and capacitor in parallel, with an active element (transistor or op-amp) to sustain oscillation
3. Replace the fixed capacitor with a variable capacitor to simulate varactor behavior

**What to observe:**
- The oscillation frequency depends on L and C: $f = 1 / (2\pi\sqrt{LC})$
- Changing the capacitance changes the frequency — this is exactly what a varactor does when you change its reverse bias voltage
- This is the basis of voltage-controlled oscillators (VCOs) used in PLLs, radio tuners, and frequency synthesizers

---
## Datasheet Connection

### AH3503 Hall Effect Sensor Datasheet

| Parameter | Typical | What to look for |
|-----------|---------|------------------|
| Supply voltage | 4.5–6.0 V | Needs regulated 5V |
| Quiescent output | 2.25–2.75 V | Output at zero field |
| Sensitivity | 1.0–1.75 mV/G | Voltage change per Gauss |
| Operating temp | -40 to 85C | |
| Response time | 3 us | Fast enough for most applications |
| Output type | Analog, ratiometric | Output scales with Vcc |

### CdS Photoresistor (GL55 series)

| Parameter | GL5528 | What to look for |
|-----------|--------|------------------|
| Light resistance (10 lux) | 8–20 kohm | Resistance at standard light |
| Dark resistance | >1 Mohm | How dark is "dark" |
| Gamma coefficient | 0.7 | Slope of log R vs log lux |
| Peak wavelength | 540 nm | Green light sensitivity |
| Rise time | 20 ms | Response to light-on |
| Fall time | 30 ms | Response to light-off |

### Varactor Diode (BB910 type)

| Parameter | Value | What to look for |
|-----------|-------|------------------|
| Capacitance at 4V | ~20 pF | Reference capacitance |
| Capacitance ratio (C1V/C8V) | ~3.0 | Tuning range |
| Max reverse voltage | 30 V | Don't exceed this |
| Series resistance | ~1 ohm | Affects Q factor |

---
## Checkpoint Questions

1. **Why do semiconductors make better Hall effect sensors than metals?
   (Hint: look at the Hall voltage equation.)**

2. **An AH3503 Hall sensor reads 3.15 V. If the sensitivity is 1.3 mV/G
   and quiescent output is 2.5 V, what is the magnetic field strength?
   Is it a North or South pole facing the sensor?**

3. **A CdS photoresistor reads 5 kohm. Using the GL5528 specs above,
   approximately how bright is the lighting?**

4. **Why can't you use a CdS photoresistor to detect a 10 kHz blinking
   LED? What would you use instead?**

5. **A varactor has C_0 = 30 pF and m = 0.5. What is its capacitance at
   10V reverse bias? (V_0 = 0.7V)**

6. **You are building an LC oscillator with a 10 uH inductor and the varactor
   from question 5. What is the oscillator frequency at 0V and at 10V
   reverse bias?**

7. **What is the difference between an NTC and PTC thermistor? Give one
   application for each.**

In [ ]:
# --- Checkpoint calculation helpers ---

# Q2: Hall sensor field calculation
V_measured = 3.15
V_q = 2.50
sensitivity = 1.3e-3  # V/Gauss
B_field = (V_measured - V_q) / sensitivity
print(f'Q2: B = ({V_measured} - {V_q}) / {sensitivity*1000} mV/G = {B_field:.0f} Gauss')
print(f'    Voltage > 2.5V, so North pole is facing the sensor')
print()

# Q5: Varactor capacitance
C_0 = 30e-12  # 30 pF
V_0 = 0.7
V_R = 10.0
m = 0.5
C_10V = C_0 / (1 + V_R / V_0) ** m
print(f'Q5: C = {C_0*1e12:.0f} pF / (1 + {V_R}/{V_0})^{m} = {C_10V*1e12:.2f} pF')
print()

# Q6: VCO frequency
L = 10e-6  # 10 uH
C_0V = C_0 / (1 + 0 / V_0) ** m  # at 0V reverse = C_0
f_0V = 1 / (2 * np.pi * np.sqrt(L * C_0V))
f_10V = 1 / (2 * np.pi * np.sqrt(L * C_10V))
print(f'Q6: At 0V:  C = {C_0V*1e12:.1f} pF, f = {f_0V/1e6:.2f} MHz')
print(f'    At 10V: C = {C_10V*1e12:.2f} pF, f = {f_10V/1e6:.2f} MHz')
print(f'    Tuning ratio: {f_10V/f_0V:.2f}:1')